In [4]:
import numpy as np
import librosa
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from tensorflow.keras.models import load_model

# ====== 설정 ======
MODEL_PATH = "multilabel_whisper_attn.keras"
THRESHOLD = 0.32
MAX_TOKEN_LENGTH = 256
SAMPLE_RATE = 16000
N_MELS = 128

# ====== 라벨 (원래 학습에 썼던 target_words를 그대로 두세요) ======
target_words = [
    "책가방","아이","현관","신발","강아지","꼬리","엄마","소파","뜨개질","아빠","신문",
    "텔레비전","테이블","로봇","라디오","창문","나무","참새","책장","책","달력",
    "시계","가족사진","강아지집","밥그릇","식탁","주전자","찻잔","과일","액자"
]
NUM_LABELS = len(target_words)

# ====== 모델 로드 ======
model = load_model(MODEL_PATH)

# ====== Whisper 로드 (학습 때와 동일) ======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)
whisper_model.eval()

# ====== 전처리 함수 ======
def load_wav_to_mel(wav_path, sr=SAMPLE_RATE, n_mels=N_MELS):
    y, _ = librosa.load(wav_path, sr=sr, mono=True)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_norm = (mel_db + 80) / 80.0              
    mel_norm = mel_norm.astype(np.float32)
    return np.expand_dims(mel_norm, axis=-1)    

@torch.no_grad()
def extract_token_ids_from_wav(wav_path):
    speech_array, _ = librosa.load(wav_path, sr=SAMPLE_RATE, mono=True)
    inputs = processor(speech_array, sampling_rate=SAMPLE_RATE, return_tensors="pt")
    input_features = inputs.input_features.to(device)

    predicted_ids = whisper_model.generate(
        input_features,
        max_new_tokens=MAX_TOKEN_LENGTH
    )
    token_ids = predicted_ids[0].cpu().numpy().astype(np.int32)

    # 패딩/자르기
    if token_ids.shape[0] < MAX_TOKEN_LENGTH:
        pad = np.zeros(MAX_TOKEN_LENGTH - token_ids.shape[0], dtype=np.int32)
        token_ids = np.concatenate([token_ids, pad], axis=0)
    else:
        token_ids = token_ids[:MAX_TOKEN_LENGTH]
    return token_ids

def prepare_inputs_for_inference(wav_path):
    mel = load_wav_to_mel(wav_path)          # (128, T, 1)
    mel = np.expand_dims(mel, axis=0)        # (1, 128, T, 1)

    token_ids = extract_token_ids_from_wav(wav_path)   # (MAX_TOKEN_LENGTH,)
    token_ids = np.expand_dims(token_ids, axis=0)      # (1, MAX_TOKEN_LENGTH)
    return mel, token_ids

# ====== 예측 및 점수 계산 (임계값 이상 개수만 점수로) ======
def score_audio(wav_path, mdl, threshold=THRESHOLD, label_names=target_words):
    X_mel, X_tok = prepare_inputs_for_inference(wav_path)
    pred = mdl.predict([X_mel, X_tok], verbose=0)[0]   # (NUM_LABELS,)
    count = int(np.sum(pred >= threshold))
    print(f"\n파일: {wav_path}")
    print(f"총점(임계값 {threshold}): {count} / {len(label_names)}")
    return count

# 사용 예시
wav_file = r"/Volumes/SSAM/project/임상data(폴더명 수정)/3019/CLAP_A/7/p_1_0.wav"
score_audio(wav_file, model, threshold=THRESHOLD)


RuntimeError: Failed to import transformers.models.whisper.modeling_whisper because of the following error (look up to see its traceback):
dlopen(/Users/kimberlyjojohirn/opt/anaconda3/envs/CLAP_PC/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib, 0x0006): Symbol not found: __ZN3tsl8internal10LogMessageC1EPKcii
  Referenced from: <D2EF42E3-3A7F-39DD-9982-FB6BCDC2853C> /Users/kimberlyjojohirn/opt/anaconda3/envs/CLAP_PC/lib/python3.10/site-packages/tensorflow-plugins/libmetal_plugin.dylib
  Expected in:     <300F849B-E9A8-3BDA-B8D9-C6617800789D> /Users/kimberlyjojohirn/opt/anaconda3/envs/CLAP_PC/lib/python3.10/site-packages/tensorflow/python/_pywrap_tensorflow_internal.so